# Impacto de la deforestación en el regimen de caudales de los rios en Colombia (TEAM 28)

## Multivariate time series forecasting

Sources :

https://towardsdatascience.com/vector-autoregressions-vector-error-correction-multivariate-model-a69daf6ab618

https://towardsdatascience.com/pairs-trading-with-cryptocurrencies-e79b4a00b015

### Libraries

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.tsa.vector_ar as var
import sklearn.metrics as skm

from statsmodels.tsa.stattools import grangercausalitytests # Granger's Causality Test

from statsmodels.graphics.tsaplots import plot_acf # Autocorrelation plot
from statsmodels.graphics.tsaplots import plot_pacf # PartialAutocorrelation plot

from sklearn.model_selection import train_test_split # Split data for machine learning

from statsmodels.tsa.stattools import adfuller # Augmented Dickey-Fuller Test (ADF Test)/unit root test
from statsmodels.tsa.stattools import coint  # Johansen Test for Cointegration (Just two data series)

from coint_johansen import coint_johansen  # Johansen Test for Cointegration more two data series
from scipy.stats import zscore # Calculates the z score of each value in the sample, relative to the sample mean and standard deviation.

from sklearn.utils import check_array # Para calculo de MAPE

from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num = None, figsize = (15, 12), dpi = 80, facecolor = 'w', edgecolor = 'k')
plt.rcParams.update({'font.size': 16, 'figure.figsize': (15, 10), 
                     'figure.max_open_warning': 200})

# machine learning: XGB
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from xgboost.sklearn import XGBRegressor # wrapper

In [ ]:
print(plt.rcParams.keys())

### Read Data

In [ ]:
macrodata = pd.read_csv('../data/matrix/matrix_consol_v2.zip')

macrodata.head(10)

In [ ]:
mcs = macrodata['mc'].unique()
mcs.sort()

print(mcs)

### Split data for machine learning algorithms

In [ ]:
temp_df = macrodata.copy()

data_train = pd.DataFrame()
data_test = pd.DataFrame()

for i in mcs:
    #train, test = train_test_split(temp_df[temp_df['mc'] == i], test_size = 0.2)
    nobs = 12 # 5% de 240
    train, test = temp_df[temp_df['mc'] == i].iloc[0:-nobs], temp_df[temp_df['mc'] == i].iloc[-nobs:]
    data_train = pd.concat([data_train, train], axis = 0)
    data_test = pd.concat([data_test, test], axis = 0)

print('Total data')
print('----------')
print(macrodata.shape)
print(macrodata.dtypes)
print()
print('data_train')
print('----------')
print(data_train.shape)
print(data_train.dtypes)
print()
print('data_test')
print('---------')
print(data_test.shape)
print(data_test.dtypes)

temp_df.head(10)

In [ ]:
data_train.head(10)

In [ ]:
data_test.head(10)

In [ ]:
dates = pd.DataFrame(pd.date_range('2018-01-01','2019-12-31' , freq='1M') - 
             pd.offsets.MonthBegin(1))
dates.columns = ['date']

dates['year'] = pd.DatetimeIndex(dates['date']).year
dates['month'] = pd.DatetimeIndex(dates['date']).month

dates.head()

## Random Forest